In [ ]:

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = ''

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [2]:
import gridfs,os


db = client["lung_xray_db"]
fs = gridfs.GridFS(db)

# Directory containing images (subfolders: NORMAL and PNEUMONIA)
base_dir = "../data/train"  # Adjust this path

def upload_images_to_mongodb(folder_path, label):
    for filename in os.listdir(folder_path):
        # Filter by image extensions (optional, to ensure only images are uploaded)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, "rb") as f:
                    file_id = fs.put(f, filename=filename, label=label)
                    print(f"Uploaded {filename} with label {label} -> ID: {file_id}")
        else:
            print(f"Skipping non-image file: {filename}")

# Upload Normal images (you can limit by number to control size)
upload_images_to_mongodb(os.path.join(base_dir, "NORMAL"), label="NORMAL")

# Upload Pneumonia images
upload_images_to_mongodb(os.path.join(base_dir, "PNEUMONIA"), label="PNEUMONIA")

Uploaded IM-0031-0001.jpeg with label NORMAL -> ID: 6801fa8dd8181481b27e8065
Uploaded IM-0025-0001.jpeg with label NORMAL -> ID: 6801fa8ed8181481b27e8068
Uploaded NORMAL2-IM-0102-0001.jpeg with label NORMAL -> ID: 6801fa8ed8181481b27e806c
Uploaded IM-0010-0001.jpeg with label NORMAL -> ID: 6801fa8fd8181481b27e806f
Uploaded IM-0046-0001.jpeg with label NORMAL -> ID: 6801fa8fd8181481b27e8072
Uploaded NORMAL2-IM-0027-0001.jpeg with label NORMAL -> ID: 6801fa8fd8181481b27e8075
Uploaded NORMAL2-IM-0033-0001.jpeg with label NORMAL -> ID: 6801fa8fd8181481b27e8077
Uploaded IM-0028-0001.jpeg with label NORMAL -> ID: 6801fa8fd8181481b27e807a
Uploaded NORMAL2-IM-0012-0001.jpeg with label NORMAL -> ID: 6801fa8fd8181481b27e807e
Uploaded IM-0009-0001.jpeg with label NORMAL -> ID: 6801fa90d8181481b27e8080
Uploaded IM-0073-0001.jpeg with label NORMAL -> ID: 6801fa90d8181481b27e8083
Uploaded IM-0067-0001.jpeg with label NORMAL -> ID: 6801fa90d8181481b27e8085
Uploaded NORMAL2-IM-0019-0001.jpeg with labe

In [2]:
# Import necessary libraries
from torchvision.datasets import ImageFolder
import torch

# Assuming you have the path to the dataset
train_data = ImageFolder('../data/train')
val_data = ImageFolder('../data/test')

# Get the labels for the training and validation datasets
train_labels = torch.tensor(train_data.targets)  # Convert to tensor if not already
val_labels = torch.tensor(val_data.targets) 

# Count the occurrences of each class in the training and validation sets
normal_count_train = (train_labels == 0).sum().item()  # Class 0: NORMAL
pneumonia_count_train = (train_labels == 1).sum().item()  # Class 1: PNEUMONIA

normal_count_val = (val_labels == 0).sum().item()  # Class 0: NORMAL
pneumonia_count_val = (val_labels == 1).sum().item()  # Class 1: PNEUMONIA

# Display the class counts
print(f"Training set - NORMAL: {normal_count_train}, PNEUMONIA: {pneumonia_count_train}")
print(f"Validation set - NORMAL: {normal_count_val}, PNEUMONIA: {pneumonia_count_val}")


Training set - NORMAL: 105, PNEUMONIA: 105
Validation set - NORMAL: 30, PNEUMONIA: 30
